In [1]:
import pandas as pd
import numpy as np
import datetime as dt

# Carga de los datasets

In [4]:
# Llamadas de Panamá
llamadas_panama=pd.read_csv('C:/Users/gurenab/Documents/Central telefonica vs registros en summa/llamadas_panama.txt', sep=",")
llamadas_panama.head()

C:\Users\gurenab\AppData\Local\Temp\ipykernel_15312\3766511407.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  llamadas_panama=pd.read_csv('C:/Users/gurenab/Documents/Central telefonica vs registros en summa/llamadas_panama.txt', sep=",")


,calldate,src,dst,lastapp,duration,billsec,disposition
0,2023-04-01 07:58:45,137,64847613,Dial,25,2,ANSWERED
1,2023-04-01 07:59:27,121,2205310,Congestion,6,4,ANSWERED
2,2023-04-01 07:59:35,121,2205310,Congestion,8,6,ANSWERED
3,2023-04-01 07:59:46,258,68159499,Dial,0,0,NO ANSWER
4,2023-04-01 07:59:49,258,63173530,AGI,2,0,ANSWERED


In [49]:
# Central telefónica
telefonos_sistema=pd.read_csv('C:/Users/gurenab/Documents/Central telefonica vs registros en summa/TELEFONOS_SISTEMA.csv',sep=';',encoding='utf-8')
telefonos_sistema.head()

,CLI_IDENTIFICACION,NUMERO
0,PA-P-8-735-1882-BB,2662864
1,PA-P-C0300008101509-CI,66751778
2,PA-P-8-192-230-SB,2254834
3,PA-P-2-86-1591-SB,64128465
4,PA-P-8-263-223-SB,62221379


# Se trabaja primero con las llamadas de Panamá

In [8]:
# La columna calldate no es del tipo Date y hora, por lo que primero se debe convertir a este formato utilizando el método to_datetime()
llamadas_panama.calldate = pd.to_datetime(llamadas_panama.calldate,format='%Y-%m-%d %H:%M:%S')
# Se crea el mes para luego agrupar por fecha
llamadas_panama['month'] = llamadas_panama['calldate'].dt.strftime('%m')
llamadas_panama.head()

,calldate,src,dst,lastapp,duration,billsec,disposition,month
0,2023-04-01 07:58:45,137,64847613,Dial,25,2,ANSWERED,04
1,2023-04-01 07:59:27,121,2205310,Congestion,6,4,ANSWERED,04
2,2023-04-01 07:59:35,121,2205310,Congestion,8,6,ANSWERED,04
3,2023-04-01 07:59:46,258,68159499,Dial,0,0,NO ANSWER,04
4,2023-04-01 07:59:49,258,63173530,AGI,2,0,ANSWERED,04


In [191]:
# Se agrupa por mes
llamadas_panama_mes = llamadas_panama.groupby(['month','dst','disposition'], as_index=False).agg({
    "month":['count'],
    "duration": ['sum']
})
llamadas_panama_mes.tail(50)

llamadas_panama_mes.columns.values

array([('dst', ''), ('disposition', ''), ('month', 'count'),
       ('duration', 'sum')], dtype=object)

# Se trabaja con los números de la central

In [65]:
# Elimino el CLI_IDENTIFICACION porque no se necesita para el análisis
columns_names = telefonos_sistema.columns.values
columns_names

# Elimino los duplicados
telefonos_sistema= telefonos_sistema.drop(columns=['CLI_IDENTIFICACION'])
telefonos_sistema = telefonos_sistema.rename(columns={'NUMERO':'dst'})
telefonos_sistema.columns
telefonos_sistema.head()

#Voy a crear una columna llamada "nada" sólo para cerciorarme de que los joins funcionan, luego no se toma en cuenta para los análisis
telefonos_sistema['nada'] = '1'
telefonos_sistema.head()

,dst,nada
0,2662864,1
1,66751778,1
2,2254834,1
3,64128465,1
4,62221379,1


# Identificar cuáles teléfonos registrados en summa no se han marcado en los últimos 3 meses (cruce de summa con la central telefónica): Aunque la cuenta esté gestionada, puede que el gestor no haya marcado todos los teléfonos.

In [122]:
# La data que tengo son llamadas que se han marcado (tienen duración y si se recibió respuesta o no, por ejemplo) y los números de la central teléfonica, por lo que sacar números que no se han marcado no es posible. En lugar de eso, voy a hacer el cruce de telefonos_sistema con llamadas_panama_mes, para obtener info de llamadas que si debieron haberse hecho.  
# Inner join: This join creates a new table which will combine table A (llamadas_panama_mes) and table B (telefonos_sistema), based on the join-predicate (the column we decide to link the data on: dst). Fusión de los dos data frames en uno que contiene los elementos comunes de ambos.

# Only use the 1st level index
llamadas_panama_mes.columns = llamadas_panama_mes.columns.get_level_values(1)
llamadas_panama_mes.head()
# Notice how left_on is now a normal string since only 1 level is used
opcion1=pd.merge(llamadas_panama_mes,telefonos_sistema,how="inner",left_on=["dst"],right_on=["dst"])
opcion1.tail()

,dst,count,sum,nada
1679568,9998121,1,9,1
1679569,9998121,1,21,1
1679570,9998225,1,17,1
1679571,9998225,1,9,1
1679572,9998477,1,39,1


# Identificar los teléfonos que aparecen como marcados en summa pero no aparecen en la central telefónica.

In [203]:
# Anti join: Returns all rows in the first data frame (llamadas_panama_mes) that do not have a matching team in the second data frame (telefonos_sistema)

#columns_names = llamadas_panama_mes.columns.values
#columns_names

# Only use the 1st level index
#llamadas_panama_mes.columns = llamadas_panama_mes.columns.get_level_values(1)
#print(llamadas_panama_mes)
# Notice how left_on is now a normal string since only 1 level is used
#opcion2=pd.merge(llamadas_panama_mes,telefonos_sistema,how="outer",left_on=["dst"],right_on=["dst"])


columns_names

#outer_join=llamadas_panama_mes.merge(telefonos_sistema, how='outer', left_on=['dst'], right_on=['dst'])
#opcion2=outer_join[~(outer_join,merge=='both')].drop('_merge',axis=1)

#outer_join.head()

#rows = []
#for i, row in telefonos_sistema.iterrows():
#    if row.dst not in llamadas_panama_mes.dst.values:
#        rows.append(row)

#pd.concat([llamadas_panama_mes.T] + rows, axis=1).T



#telefonos_sistema.columns.values
#llamadas_panama_mes.columns.values


array([('dst', ''), ('disposition', ''), ('month', 'count'),
       ('duration', 'sum')], dtype=object)